# Рабочая тетрадь к занятию «Mediapipe. Модуль Hands»

## Задание №1
Переделайте программу так, чтобы она брала изображение с камеры и обрабатывала видеопоток

In [ ]:
# 0. Импортируем необходимое
# из библиотек OpenCV и MediaPipe
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# и собственную функцию для рисования
from draw_landmarks import draw_landmarks_on_image

In [ ]:
# 1. Создаем и настраиваем детектор
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [ ]:
# 2 Oткрываем видеопоток
cap = cv2.VideoCapture(0)
if not cap.isOpened():
  raise IOError("Ошибка видеозахвата!")

key = None
# завершить программу можно клавишей ESC
while cap.isOpened() and key != 27:
    # 2. Подготовливаем изображение
    ret, frame = cap.read()
    if ret != True:
        continue
    # Отражаем кадр по горизонтали
    frame = frame[:, ::-1, :]
    # Перeводим его в формат Mediapipe-изображений
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    # 3. Самое главное: РАСПОЗНАЕМ
    detection_result = detector.detect(image)

    # Отрисовываем результат распознавания
    annotated_image = draw_landmarks_on_image(image, detection_result)
    cv2.imshow("Result", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

    # Задерживаем программу до нажатия на кнопку
    key = cv2.waitKey(1)
cv2.destroyAllWindows()

## Задание №2
Допишите функцию, которая принимает имя картинки (без папки, только имя) и определяет, есть ли рука на этой картинке

![](https://github.com/vv73/HandsMediapipe/raw/master/_common_res/task2.png)

In [ ]:
def hand_detected(pic_name):
    # 2. Подготовливаем изображение
    cv_mat = cv2.cvtColor(cv2.imread(pic_name), cv2.COLOR_RGB2BGR)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv_mat)
    # 3. Самое главное: РАСПОЗНАЕМ
    detection_result = detector.detect(image)
    result = False
    if len(detection_result.handedness) > 0:
        result = True
    return result

Запустите ячейку выше, а затем ниже, чтобы протестировать свое решение

In [ ]:
# У учащихся просто "test/..."
if not hand_detected("../test/no_hand1.png") and hand_detected("../test/hand1.png"):
    print("OK")
else:
    print("Что-то неправильно...")

## Задание №3

Напишите программу, которая раскрашивает подушечки пальцев в разные цвета.

In [1]:
# 0. Импортируем необходимое
# из библиотек OpenCV и MediaPipe
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# и собственную функцию для рисования
from draw_landmarks import draw_landmarks_on_image

# 1. Создаем и настраиваем детектор
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# 2 открываем видеопоток
cap = cv2.VideoCapture(0)
if not cap.isOpened():
  raise IOError("Ошибка видеозахвата!")

# Определяем цвета
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]
# Индексы подушечек (вообще, есть закономерность, но списком понятнее)
fingers = [4, 8, 12, 16, 20]

key = None
# завершить программу можно клавишей ESC
while cap.isOpened() and key != 27:
    # 2. Подготовливаем изображение
    ret, frame = cap.read()
    if ret != True:
        continue
    # Отражаем кадр по горизонтали
    frame = frame[:, ::-1, :]
    # Перeводим его в формат Mediapipe-изображений
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    # 3. Самое главное: РАСПОЗНАЕМ
    detection_result = detector.detect(image)

    if len(detection_result.hand_landmarks) > 0:
        for i in range(5):
        # нужно умножить координаты на размеры картинки
            x_tip = int(detection_result.hand_landmarks[0][fingers[i]].x *
                        image.width)
            y_tip = int(detection_result.hand_landmarks[0][fingers[i]].y *
                        image.height)
            # рисуем кружок
            cv2.circle(frame, (x_tip, y_tip), 10, colors[i], -1)
    # переводим обратно в BGR и показываем
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("Example", frame)

     #Задерживаем программу до нажатия на кнопку
    key = cv2.waitKey(1)
cv2.destroyAllWindows()

## Задание №3 (*)

Напишите программу, которая определяет по видео, когда "видит" кулак, например рисует окружность вокруг кисти разного цвета, когда кулак сжат и разжат.

In [ ]:
# 0. Импортируем необходимое
# из библиотек OpenCV и MediaPipe
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

def get_points(landmark, shape):
    points = []
    for mark in landmark:
        points.append([mark.x * shape[1], mark.y * shape[0]])
    return np.array(points, dtype=np.int32)

def palm_size(landmark, shape):
    x1, y1 = landmark[0].x * shape[1], landmark[0].y * shape[0]
    x2, y2 = landmark[5].x * shape[1], landmark[5].y * shape[0]
    return ((x1 - x2)**2 + (y1 - y2) **2) **.5

# 1. Создаем и настраиваем детектор
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q') or not ret:
        break
    flipped = np.fliplr(frame)
    # переводим его в формат RGB для распознавания
    cv_mat = cv2.cvtColor(flipped, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv_mat)
    # Распознаем
    results = detector.detect(image)
    # Рисуем распознанное, если распозналось
    if len(results.hand_landmarks) > 0:
         #cv2.drawContours(flipped, [get_points(results.hand_landmarks[0], cv_mat.shape)], 0, (255, 0, 0), 2)
         (x, y), r = cv2.minEnclosingCircle(get_points(results.hand_landmarks[0], cv_mat.shape))
         ws = palm_size(results.hand_landmarks[0], cv_mat.shape)
         print(2 * r / ws)

         # Коэффициент 1.3 следует подобрать
         if 2 * r / ws > 1.3:
             cv2.circle(cv_mat,(int(x), int(y)), int(r), (0, 0, 255), 2)
         else:
             cv2.circle(cv_mat,(int(x), int(y)), int(r), (0, 255, 0), 2)


    # переводим в BGR и показываем результат
    res_image = cv2.cvtColor(cv_mat, cv2.COLOR_RGB2BGR)
    cv2.imshow("Hands", res_image)

# освобождаем ресурсы
detector.close()